# Basic Chatbot
- This is a simple chatbot that answers basic questions
- It is also able to do basic tasks


In [1]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("HUGGING_FACE_API_KEY")
_set_env("TAVILY_API_KEY")

## Populating the database
- The following script populates the sqlite database for the vehicle information

In [2]:
import os
import shutil
import sqlite3

import pandas as pd
import requests

In [8]:
vehicledb_url = "https://raw.githubusercontent.com/AdityaUnal/dbRentalShop/refs/heads/main/vehicledb.sqlite?token=GHSAT0AAAAAADECLLHCGSRZNWZ3D7QVVSP42CI3TEQ"
local_file = "vehicledb.sqlite"
overwrite = False
if overwrite or not os.path.exists(local_file):
    print(f"Downloading {vehicledb_url} to {local_file}")
    response = requests.get(vehicledb_url)
    with open(local_file, "wb") as f:
        f.write(response.content)
else:
    print(f"Using existing {local_file}")

def get_vehicle_data():
    try:
        vehicle_id = input("Vehicle ID: ").strip()
        if not vehicle_id:
            raise ValueError("Vehicle ID is required.")

        vehicle_name = input("Vehicle Name: ").strip()
        if not vehicle_name:
            raise ValueError("Vehicle Name is required.")

        type = input("Vehicle Type (Car/Bike/Scooty): ").strip()
        vehicle_type = type if type else "Scooty"

        wheel_input = input("Wheel Count (e.g. 2, 4): ").strip()
        vehicle_wheel_count = int(wheel_input) if wheel_input else 2  # default to 2 wheels

        gear_count = input("Gear Count: ").strip()
        vehicle_gear_count = gear_count if gear_count else "manual"

        price_day_input = input("Price per Day: ").strip()
        vehicle_price_per_day = float(price_day_input) if price_day_input else 500.0

        price_hour_input = input("Price per Hour: ").strip()
        vehicle_price_per_hour = float(price_hour_input) if price_hour_input else 50.0

        vehicle_description = input("Description: ").strip() or "No description available"

        mileage_input = input("Mileage (km/l): ").strip()
        vehicle_mileage = float(mileage_input) if mileage_input else 30.0


        fuel_type = input("Fuel Type: ").strip()
        vehicle_fuel_type = fuel_type if fuel_type else "petrol"

        transmission = input("Transmission (manual/auto/none): ").strip()
        vehicle_transmission = transmission if transmission else "manual"

        seating_input = input("Seating Capacity: ").strip()
        vehicle_seating_capacity = int(seating_input) if seating_input else 2


        return {
            "vehicle_id": vehicle_id,
            "vehicle_name": vehicle_name,
            "vehicle_type": vehicle_type,
            "vehicle_wheel_count": vehicle_wheel_count,
            "vehicle_gear_count": vehicle_gear_count,
            "vehicle_price_per_day": vehicle_price_per_day,
            "vehicle_price_per_hour": vehicle_price_per_hour,
            "vehicle_description": vehicle_description,
            "vehicle_mileage": vehicle_mileage,
            "vehicle_fuel_type": vehicle_fuel_type,
            "vehicle_transmission": vehicle_transmission,
            "vehicle_seating_capacity": vehicle_seating_capacity,
        }

    except ValueError as ve:
        print(f"\n Invalid input: {ve}")
        return None
    except Exception as e:
        print(f"\n Unexpected error: {e}")
        return None


def add_vehicle(file,vehicle_data):
    conn = sqlite3.connect(file)
    cursor = conn.cursor()
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS vehicles (
        vehicle_id TEXT PRIMARY KEY,
        vehicle_name TEXT,
        vehicle_type TEXT
        vehicle_wheel_count INTEGER,
        vehicle_gear_count TEXT,
        vehicle_price_per_day REAL,
        vehicle_price_per_hour REAL,
        vehicle_description TEXT,
        vehicle_mileage REAL,
        vehicle_fuel_type TEXT,
        vehicle_transmission TEXT,
        vehicle_seating_capacity INTEGER,
        vehicle_user_rating REAL,
        vehicle_user_feedback TEXT
        )
    """)
    columns = ', '.join(vehicle_data.keys())
    placeholders = ', '.join(['?'] * len(vehicle_data))
    values = tuple(vehicle_data.values())
    cursor.execute(
        f"INSERT OR IGNORE INTO vehicles ({columns}) VALUES ({placeholders})",
        values
    )
    conn.commit()
    conn.close()

while True:
    again = input("Do you want to add a vehicle? (yes/no): ").strip().lower()
    if again != "yes":
        break
    vehicle_data = get_vehicle_data()
    add_vehicle(local_file,vehicle_data)



Using existing vehicledb.sqlite


DatabaseError: file is not a database

Displays the existing vehicles in the db

In [9]:
from tabulate import tabulate

conn = sqlite3.connect("vehicledb.sqlite")
df = pd.read_sql("SELECT * FROM vehicles", conn)
conn.close()

print(tabulate(df, headers='keys', tablefmt='psql'))  # or 'fancy_grid', 'grid'


DatabaseError: Execution failed on sql 'SELECT * FROM vehicles': file is not a database

## Tools
- We define the tools of our chatbot
- These tools will help in vehicle recommendation, repairs, user feedback of the vehicle

### Vehicle Recommendation
- Based on the user's travel destination, travel budget, weather, the feedback on vehicle